# Reto Limpieza del Conjunto de Datos

### Equipo
Diego Díaz,
Carlos Ortega,
Eduardo González,
Jesús Miranda,
Hilda Beltrán

## Extract data

In [12]:
# Import necessary libraries to extract, transform and visualize data
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import re

In [13]:
# Path where files are stored
DATA_PATH = 'C:/Users/hilda/Documents/Codigos IA/Reto/Datasets'

In [14]:
# Create a DataFrame from each dataset
df_train = pd.read_csv(os.path.join(DATA_PATH,'train.csv'))
df_oil = pd.read_csv(os.path.join(DATA_PATH,'oil.csv'))
df_holiday = pd.read_csv(os.path.join(DATA_PATH,'holidays_events.csv'))
df_stores = pd.read_csv(os.path.join(DATA_PATH,'stores.csv'))
df_transactions = pd.read_csv(os.path.join(DATA_PATH,'transactions.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH,'test.csv'))

## Cleaning data

### Looking for NaN values

In [15]:
# Look for NaN values in the DataFrames
henan = df_holiday.isna().sum().sum()
oilnan = df_oil.isna().sum().sum()
storesnan = df_stores.isna().sum().sum()
testnan = df_test.isna().sum().sum()
trainnan = df_train.isna().sum().sum()
transnan = df_transactions.isna().sum().sum()

print('NaN train:', trainnan)
print('NaN test:', testnan)
print('NaN oil:', oilnan)
print('NaN holidays events:', henan)
print('NaN stores:', storesnan)
print('NaN transactions:', transnan)

NaN train: 0
NaN test: 0
NaN oil: 43
NaN holidays events: 0
NaN stores: 0
NaN transactions: 0


In [16]:
# Since there are 43 NaN values in df_oil, we'll analyze which Data Imputation method works best for this case
df_oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [17]:
# Trying to interpolate data with a linear method
df_oil_i = df_oil.copy()
df_oil_i = df_oil_i.interpolate(method = 'linear')
df_oil_i.loc[0, 'dcoilwtico'] = df_oil_i.dcoilwtico[1]
df_oil_i.describe()

,dcoilwtico
count,1218.000000
mean,67.682742
std,25.632933
min,26.190000
25%,46.422500
50%,53.185000
75%,95.685000
max,110.620000


In [18]:
# By replacing NaN values with the mean of the price attribute we get a smaller standard deviation
# We will continue working with this DataFrame
df_oil.dcoilwtico.fillna(df_oil['dcoilwtico'].mean(), inplace = True)
df_oil.describe()

,dcoilwtico
count,1218.000000
mean,67.714366
std,25.173606
min,26.190000
25%,46.700000
50%,53.925000
75%,95.332500
max,110.620000


### Looking for and replacing special characters and numbers

In [19]:
# By viewing the holidays_events file, we notice there were some special characters in the description column.
# Therefore, in order to get a single categorization for each event in the description, we will ignore this distinction.
# We're also ignoring the word 'Puente' from the description column.
df_holiday.description = df_holiday.description.str.replace('-', '').str.replace('+', '').str.replace('\d+', '', regex=True).str.replace('Puente ', '')

## Data Transformation

In [20]:
# The holidays_events file has a category for Holiday and Transfer, as well as a transferred column.
# Since we want the day in which the holiday was celebrated, we'll get this info by comparing the date for both categories.
holiday = pd.DataFrame()
h = df_holiday[(df_holiday.type == 'Holiday') & (df_holiday.transferred == True)]
t = df_holiday[(df_holiday.type == 'Transfer')]
holiday = h.drop('transferred', axis = 1).reset_index(drop = True)
holiday['date'] = t.date.reset_index(drop = True)
df_holiday = df_holiday[(df_holiday.transferred == False) & (df_holiday.type != 'Transfer')].drop('transferred', axis = 1)
df_holiday.describe()

,date,type,locale,locale_name,description
count,326,326,326,326,326
unique,290,5,3,24,55
top,2014-06-25,Holiday,National,Ecuador,Navidad
freq,4,209,158,158,38


In [21]:
pd.concat([df_holiday, holiday], ignore_index=True)
df_holiday['type'] = np.where(df_holiday['type'] == 'Additional', 'Holiday', df_holiday['type'])
df_holiday['type'] = np.where(df_holiday['type'] == 'Bridge', 'Holiday', df_holiday['type'])
df_holiday.describe()

,date,type,locale,locale_name,description
count,326,326,326,326,326
unique,290,3,3,24,55
top,2014-06-25,Holiday,National,Ecuador,Navidad
freq,4,265,158,158,38


In [ ]:
# Add a column in train to get the correlation between sales and the item's family
df_train['sales-family'] = df_train.sales.corr(df_train.family)
df_train

In [ ]:
print(df_train.sales.corr(df_train.family))